In [2]:
%pylab inline
import mailbox
import pandas as pd
from tqdm import tqdm

Populating the interactive namespace from numpy and matplotlib


In [148]:
mb = mailbox.mbox('data/frichti.mbox')

# Get the first message
#msg = mb[653]
msg = mb[345]

# Get the first message's payload
payload = msg.get_payload()

# Get the subject converted to the right encoding (french)
# The subject looks like this b'=?utf-8?Q?Frigo=20vide=20=3F=20Prenez=20les=20choses=20en=20main=20=F0=9F=9B=92?='
# Convert to plain text
import quopri
subject = quopri.decodestring(msg['subject'].encode('utf-8')).decode('utf-8')
# Now it looks like this: =?utf-8?Q?Frigo vide ? Prenez les choses en main 🛒?
subject = subject.replace('=?utf-8?Q?', '').replace('?=', '')
print("Subject: {}".format(subject))


# Convert the payload to a string with the right encoding (french)
payload[0].get_payload(decode=True).decode('utf-8')

# Remove urls
import re
def remove_urls(text):
    # Remove urls and surrounding parentheses or brackets
    return re.sub(r'\(?\[?(https?:\/\/[^\s\)]+)\]?\)?', '', text)
    
def remove_end(text):
    # remove everything after multiples = or - or _ or **
    # or after Twitter, Facebook, Instagram, LinkedIn, Youtube, Pinterest, Snapchat
    end_pattern = re.compile(r'={2,}|-{2,}|_{2,}|\*{2,}|Twitter|Facebook|Instagram|LinkedIn|Youtube|Pinterest|Snapchat')
    # Find the first match
    match = end_pattern.search(text)
    if match:
        # Remove everything after the match
        return text[:match.start()]
    else:
        return text
    
def remove_blank_lines(text):
    return re.sub(r'\n\s*\n', '\n\n', text)

def remove_short_lines(text):
    # Remove lines with less than 3 words
    # print matches
    pattern = r'(?m)^\s*\S+(?:\s+\S+){0,2}\s*$'
    return re.sub(pattern, '', text, flags=re.MULTILINE)

def remove_product(text):
    #TODO this prompt is very specific to frichti, which put an all cap sentence before the products
    # find an all cap line surrounded by \n in a french text
    # do not match blank lines
    #pattern = r'(?m)^\s*[A-ZÀÂÊÔÛÇÉÈÏÎËËÏÛÙÆŒ][A-ZÀÂÊÔÛÇÉÈÏÎËËÏÛÙÆŒ\s\-\']+\s*$'
    emojis = r'\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF\U0001F1E0-\U0001F1FF'
    pattern = """(?m)^\s*[A-ZÀÂÊÔÛÇÉÈÏÎËËÏÛÙÆŒ'.,!€$?\-{}][A-ZÀÂÊÔÛÇÉÈÏÎËËÏÛÙÆŒ\s'.,!?€$\-{}]+\s*$""".format(emojis, emojis)
    # match line with no lower case letter, but not blank lines
    #pattern = r'\n[^a-zàâêôûçéèïîëëïûùæœ][A-ZÀÂÊÔÛÇÉÈÏÎËËÏÛÙÆŒ]+[^a-zàâêôûçéèïîëëïûùæœ]\n'
    # Find a match
    match = re.search(pattern, text)
    if match:
        #print("Match: {}".format(repr(match.group())))
        # Remove everything after the match
        return text[:match.start()]
    else:
        #print("No match")
        return text


    


def process_text(text):
    # Remove urls
    text = remove_urls(text)
    # Remove end
    text = remove_end(text)
    # Remove products
    #text = remove_short_lines(text)
    # Remove products
    text = remove_product(text)
    # Remove blank lines
    text = remove_blank_lines(text)
    return text
# print with urls removed
print(payload[0].get_payload(decode=True).decode('utf-8'))
print("------ Processed text ------")
print("--Subject: {}".format(subject))
print(process_text(payload[0].get_payload(decode=True).decode('utf-8')))


Subject: [FRICHTI DEAL$] Vos courses bien-être 💚?
Découvrez nos produits feel good en promo !

https://frichti.app.link/lk7k7rQ4fob?goal=0_b8176c28ad-51bbf7dd24-176650961&mc_cid=51bbf7dd24&mc_eid=UNIQID
https://frichti.app.link/lk7k7rQ4fob?goal=0_b8176c28ad-51bbf7dd24-176650961&mc_cid=51bbf7dd24&mc_eid=UNIQID
Des promos “healthy” !

Cette semaine, Frichti Deal$ = bien-être.
Dans votre assiette et votre portefeuille.

Traversez votre journée, du petit-déjeuner au dîner, avec légèreté.
Dès la matinée, avec jusqu’à -25% sur votre bol de muesli, et votre kiwi de réveil.
On ne s’arrête pas en si bon chemin, avec jusqu’à -50% sur une sélection de velouté de légumes.
En ce moment, profitez de 20€ offerts sur votre 1ère commande avec le code* :
RENCONTRE20
I FEEL GOOD (https://frichti.app.link/lk7k7rQ4fob?goal=0_b8176c28ad-51bbf7dd24-176650961&mc_cid=51bbf7dd24&mc_eid=UNIQID)
https://frichti.app.link/lk7k7rQ4fob?goal=0_b8176c28ad-51bbf7dd24-176650961&mc_cid=51bbf7dd24&mc_eid=UNIQID
https://fri

In [149]:
# Gather all the content in a pd.DataFrame
df = pd.DataFrame(columns=['date', 'content'])
mb = mailbox.mbox('data/frichti.mbox')
for msg in tqdm(mb):
    # Get subject
    subject = quopri.decodestring(msg['subject'].encode('utf-8')).decode('utf-8')
    subject = subject.replace('=?utf-8?Q?', '').replace('?=', '')
    # Get content
    payload = msg.get_payload()
    # check if type of payload is str
    if not isinstance(payload, str):
        content = payload[0].get_payload(decode=True)
        if not content is None:
            content = process_text(content.decode('utf-8'))
    else:
        print(payload)
    df = df.append({'date': msg['date'], "subject": subject, 'content': content}, ignore_index=True)

 31%|███       | 234/759 [00:00<00:02, 259.55it/s]

<!doctype html>
    <html xmlns=3D"http://www.w3.org/1999/xhtml" xmlns:v=3D"urn:schemas-mic=
rosoft-com:vml" xmlns:o=3D"urn:schemas-microsoft-com:office:office">
      <head>
        <title>
          X-ENTREPRENEURSHIP // We have some news for you !
        </title>
        <!--[if !mso]><!-- -->
        <meta http-equiv=3D"X-UA-Compatible" content=3D"IE=3Dedge">
        <!--<![endif]-->
        <meta http-equiv=3D"Content-Type" content=3D"text/html; charset=3DU=
TF-8">
        <meta name=3D"viewport" content=3D"width=3Ddevice-width, initial-sc=
ale=3D1">
        <style type=3D"text/css">
          #outlook a { padding:0; }
          .ReadMsgBody { width:100%; }
          .ExternalClass { width:100%; }
          .ExternalClass * { line-height:100%; }
          body { margin:0;padding:0;-webkit-text-size-adjust:100%;-ms-text-=
size-adjust:100%; }
          table, td { border-collapse:collapse;mso-table-lspace:0pt;mso-tab=
le-rspace:0pt; }
          img { border:0;height:auto;line-heigh

 52%|█████▏    | 397/759 [00:01<00:01, 264.01it/s]

<!doctype html>
    <html xmlns=3D"http://www.w3.org/1999/xhtml" xmlns:v=3D"urn:schemas-mic=
rosoft-com:vml" xmlns:o=3D"urn:schemas-microsoft-com:office:office">
      <head>
        <title>
          X-ENTREPRENEURSHIP // We have some news for you !
        </title>
        <!--[if !mso]><!-- -->
        <meta http-equiv=3D"X-UA-Compatible" content=3D"IE=3Dedge">
        <!--<![endif]-->
        <meta http-equiv=3D"Content-Type" content=3D"text/html; charset=3DU=
TF-8">
        <meta name=3D"viewport" content=3D"width=3Ddevice-width, initial-sc=
ale=3D1">
        <style type=3D"text/css">
          #outlook a { padding:0; }
          .ReadMsgBody { width:100%; }
          .ExternalClass { width:100%; }
          .ExternalClass * { line-height:100%; }
          body { margin:0;padding:0;-webkit-text-size-adjust:100%;-ms-text-=
size-adjust:100%; }
          table, td { border-collapse:collapse;mso-table-lspace:0pt;mso-tab=
le-rspace:0pt; }
          img { border:0;height:auto;line-heigh

 67%|██████▋   | 507/759 [00:01<00:00, 269.47it/s]

<!DOCTYPE html>=0A<html style=3D"font-family: 'ProximaNova', sans-serif;">=
=0A<head>=0A<meta http-equiv=3D"Content-Type" content=3D"text/html; charset=
=3DUTF-8">=0A<style>@media only screen and (max-width: 620px) {=0A  table[c=
lass=3Dbody] h1 {=0A    font-size: 28px !important; margin-bottom: 10px !im=
portant;=0A  }=0A  table[class=3Dbody] p {=0A    font-size: 16px !important=
;=0A  }=0A  table[class=3Dbody] ul {=0A    font-size: 16px !important;=0A  =
}=0A  table[class=3Dbody] ol {=0A    font-size: 16px !important;=0A  }=0A  =
table[class=3Dbody] td {=0A    font-size: 16px !important;=0A  }=0A  table[=
class=3Dbody] span {=0A    font-size: 16px !important;=0A  }=0A  table[clas=
s=3Dbody] a {=0A    font-size: 16px !important;=0A  }=0A  table[class=3Dbod=
y] .wrapper {=0A    padding: 10px !important;=0A  }=0A  table[class=3Dbody]=
 .article {=0A    padding: 10px !important;=0A  }=0A  table[class=3Dbody] .=
content {=0A    padding: 0 !important;=0A  }=0A  table[class=3Dbody] .conta=


 78%|███████▊  | 590/759 [00:02<00:00, 267.00it/s]

<!doctype html>
    <html xmlns=3D"http://www.w3.org/1999/xhtml" xmlns:v=3D"urn:schemas-mic=
rosoft-com:vml" xmlns:o=3D"urn:schemas-microsoft-com:office:office">
      <head>
        <title>
          X-ENTREPRENEURSHIP // HERE WE GO AGAIN !
        </title>
        <!--[if !mso]><!-- -->
        <meta http-equiv=3D"X-UA-Compatible" content=3D"IE=3Dedge">
        <!--<![endif]-->
        <meta http-equiv=3D"Content-Type" content=3D"text/html; charset=3DU=
TF-8">
        <meta name=3D"viewport" content=3D"width=3Ddevice-width, initial-sc=
ale=3D1">
        <style type=3D"text/css">
          #outlook a { padding:0; }
          .ReadMsgBody { width:100%; }
          .ExternalClass { width:100%; }
          .ExternalClass * { line-height:100%; }
          body { margin:0;padding:0;-webkit-text-size-adjust:100%;-ms-text-=
size-adjust:100%; }
          table, td { border-collapse:collapse;mso-table-lspace:0pt;mso-tab=
le-rspace:0pt; }
          img { border:0;height:auto;line-height:100%; o

100%|██████████| 759/759 [00:02<00:00, 261.74it/s]


In [150]:
for i in range(0, len(df)):
    print("Date: {}".format(df['date'][i]))
    print("Subject: {}".format(df['subject'][i]))
    print("Content: {}".format(df['content'][i]))
    print("-----------------")

Date: Fri,  2 Dec 2022 08:05:02 +0000
Subject: On vous sert un petit verre ? 🍷?
Content: Découvrez nos ventes privées sur le vin

C'est votre jour de chance ! 😏

Ils s'appellent Sancerre bio, Saint-Émilion, ou encore Les Darons. Ils viennent d’un peu partout en 🇫🇷 et sont en ce moment à prix réduit. 🤩

Découvrez-les de plus près ! 🍇

On commence par le vin blanc avec notre Sancerre Fleuriet & Fils en 100% sauvignon, sa robe est jaune pâle et ses arômes évoquent le pamplemousse ainsi que les fleurs blanches.

Pour le vin rouge, vous avez le choix entre un Saint-Émilion grand cru 2012 AOP au bel équilibre et un superbe magnum Les Darons (en AOP également) au nez fruité et puissant en bouche à 12,80€ SEULEMENT !! 🔥

-----------------
Date: Sat, 10 Dec 2022 11:44:22 +0000
Subject: Merci d’avoir commandé chez Frichti !?
Content:     Merci d’avoir commandé chez Frichti !
<

-----------------
Date: Mon, 26 Dec 2022 17:04:43 +0000
Subject: Nicolas Feuillatte s'invite à vos réveillons !?
Conten

In [151]:
import numpy as np
# Clean 
print("Number of rows before: {}".format(len(df)))
# Remove all the rows with a None value
df = df.dropna()

# How many rows have a subjet which contains the string "propos de votre commande"
print("Number of rows with subject containing 'propos de votre commande': {}".format(len(df[df['subject'].str.contains('propos de votre commande')])))
# Remove all the rows with the subject "À propos de votre commande Frichti?"
df = df[~df['subject'].str.contains('propos de votre commande')]

# Remove all the content with less than 30 words
df['content'] = df['content'].apply(lambda x: x if len(x.split()) > 30 else None)

df = df.dropna()
print("Number of rows after: {}".format(len(df)))

# Remove all the content if it contains the word "incident" or "remboursement" or "Leo wrote:"
df['content'] = df['content'].apply(lambda x: x if not "incident" in x.lower() else None)
df = df.dropna()
df['content'] = df['content'].apply(lambda x: x if not "remboursement" in x.lower() else None)
df = df.dropna()
df['content'] = df['content'].apply(lambda x: x if not "leo wrote:" in x.lower() else None)
df = df.dropna()


# reindex
df = df.reset_index(drop=True)

print("Number of rows after: {}".format(len(df)))


Number of rows before: 759
Number of rows with subject containing 'propos de votre commande': 26
Number of rows after: 640
Number of rows after: 624


In [152]:
# Using OpenAI API, add a column to the dataframe with the summary of the content
import openai
api_key = ""
openai.api_key = api_key

summaries = []

for i in tqdm(range(0, len(df))):
    prompt = "Résume rapidement le contenu du mail suivant. \nObject: {}\nContenu: {}\nRésumé: Une newsletter de Frichti concernant".format(df['subject'][i], df['content'][i])
    response = openai.Completion.create(
      engine="text-davinci-002",
      max_tokens=50,
      prompt=prompt,
    )
    summaries.append(response['choices'][0]['text'])
    #print(response['choices'][0]['text'])

100%|██████████| 624/624 [16:43<00:00,  1.61s/it]


In [155]:
df["summary"] = summaries
df["summary"] = "Une newsletter de Frichti concernant" + df["summary"]

In [156]:
# Add the summaries to the dataframe
#df['summary'] = summaries

# Save the dataframe to a csv file
df.to_csv('data/frichti.csv', index=False)

In [157]:
df.to_csv('data/frichti.csv', index=False)
# Save also with pickle
df.to_pickle('data/frichti.pkl')

In [ ]:
# Fine-tune the model on the Frichti dataset with the OpenAI API
import openai

# Create the dataset
dataset = "frichti"
# Use the separator \n\n###\n\n at the end of the prompt
# Add END at the end of the content
# Use 1 epoch and a lower learning rate

#df_clean = df[[""]]


In [158]:
df

,date,content,subject,summary
0,"Fri, 2 Dec 2022 08:05:02 +0000",Découvrez nos ventes privées sur le vin\r\n\nC...,On vous sert un petit verre ? 🍷?,Une newsletter de Frichti concernant une vente...
1,"Mon, 26 Dec 2022 17:04:43 +0000",Pas besoin de rajouter un couvert. Sortez just...,Nicolas Feuillatte s'invite à vos réveillons !?,Une newsletter de Frichti concernant le champa...
2,"Sat, 17 Dec 2022 09:00:11 +0000",En voilà un bon programme !\r\n\nLa semaine co...,Samedi raclette !?,Une newsletter de Frichti concernant les produ...
3,"Mon, 2 Jan 2023 09:18:16 +0000",Et bonne année !\r\n\nBonne année et bon goûte...,Découvrez nos pépites pour le goûter 🍰?,Une newsletter de Frichti concernant leurs pro...
4,"Tue, 25 Oct 2022 18:01:03 +0000",15€ offerts sur les ventes The Bradery ! 🛍\r\n...,💖 Frichti x The Bradery?,Une newsletter de Frichti concernant une promo...
...,...,...,...,...
619,"Wed, 4 Nov 2020 15:44:37 +0000",Des produits 100% clean 100% bons livrés chez ...,Le panier de courses que vous aviez réclamé 🤩?,Une newsletter de Frichti concernant les produ...
620,"Tue, 11 Oct 2022 07:37:27 +0000","\r\n\nSous l'océan, sous l'océan...\r\n\nNotre...",3€ offerts sur votre déjeuner ! 🐟?,Une newsletter de Frichti concernant une promo...
621,"Wed, 18 Nov 2020 10:01:14 +0000",Formules 100% cheesy pour 2 personnes\r\n\nAu ...,"Allez chercher votre conjoint, c’est important.?",Une newsletter de Frichti concernant une offre...
622,"Sat, 5 Feb 2022 09:30:20 +0000","Le brunch à petit prix, c’est chez Frichti\r\n...",On brunch et on débranche 🔌?,Une newsletter de Frichti concernant un brunch...


In [159]:
# Rename summary as "prompt" and use Subject + Content as "completion"
# Add END at the end of the content
# Use the separator \n\n###\n\n at the end of the prompt

df_clean = pd.DataFrame(columns=['prompt', 'completion'])
for i in tqdm(range(0, len(df))):
    prompt = "{}\n\n###\n\n".format(df['summary'][i])
    content = "Objet: {} \n Contenu: {}".format(df['subject'][i], df['content'][i])
    completion = " {}\nEND".format(content)
    df_clean = df_clean.append({'prompt': prompt, 'completion': completion}, ignore_index=True)


100%|██████████| 624/624 [00:01<00:00, 591.73it/s]


In [160]:
df_clean

,prompt,completion
0,Une newsletter de Frichti concernant une vente...,Objet: On vous sert un petit verre ? 🍷? \n Co...
1,Une newsletter de Frichti concernant le champa...,Objet: Nicolas Feuillatte s'invite à vos réve...
2,Une newsletter de Frichti concernant les produ...,Objet: Samedi raclette !? \n Contenu: En voil...
3,Une newsletter de Frichti concernant leurs pro...,Objet: Découvrez nos pépites pour le goûter 🍰...
4,Une newsletter de Frichti concernant une promo...,Objet: 💖 Frichti x The Bradery? \n Contenu: 1...
...,...,...
619,Une newsletter de Frichti concernant les produ...,Objet: Le panier de courses que vous aviez ré...
620,Une newsletter de Frichti concernant une promo...,Objet: 3€ offerts sur votre déjeuner ! 🐟? \n ...
621,Une newsletter de Frichti concernant une offre...,"Objet: Allez chercher votre conjoint, c’est i..."
622,Une newsletter de Frichti concernant un brunch...,Objet: On brunch et on débranche 🔌? \n Conten...


In [161]:
df_clean.to_json("data/frichti_clean.jsonl", orient='records', lines=True)

In [162]:
!openai tools fine_tunes.prepare_data -f frichti_clean.jsonl -q

Analyzing...

- Your file contains 624 prompt-completion pairs
- There are 2 duplicated prompt-completion sets. These are rows: [114, 447]
- All prompts end with suffix `\n\n###\n\n`
- All prompts start with prefix `Une newsletter de Frichti concernant `. Fine-tuning doesn't require the instruction specifying the task, or a few-shot example scenario. Most of the time you should only add the input data into the prompt, and the desired output into the completion
- All completions start with prefix ` Objet: `. Most of the time you should only add the output data into the completion, without any prefix
- All completions end with suffix `\nEND`

Based on the analysis we will perform the following actions:
- [Recommended] Remove 2 duplicate rows [Y/n]: Y
- [Recommended] Remove prefix `Une newsletter de Frichti concernant ` from all prompts [Y/n]: Y
/Users/leo/opt/anaconda3/lib/python3.8/site-packages/openai/validators.py:279: SettingWithCopyWarning: 
A value is trying to be set on a copy of 

In [151]:
# Remove duplicated rows [116, 392, 405, 483, 593, 624, 629, 726, 739]
#df_clean = df_clean.drop_duplicates(subset=['prompt', 'completion'], keep='first')

In [3]:
import os
os.environ["OPENAI_API_KEY"] = api_key

NameError: name 'api_key' is not defined

In [166]:
!openai wandb sync

wandb: Currently logged in as: leogrin. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.13.10
wandb: Run data is saved locally in /Users/leo/VSCProjects/newletters/wandb/run-20230217_135404-ft-CQbXPsrw8uLqcGXMtbCoSQTX
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run ft-CQbXPsrw8uLqcGXMtbCoSQTX
wandb: ⭐️ View project at https://wandb.ai/leogrin/GPT-3
wandb: 🚀 View run at https://wandb.ai/leogrin/GPT-3/runs/ft-CQbXPsrw8uLqcGXMtbCoSQTX
^C

wandb: Waiting for W&B process to finish... (success).


In [168]:
# Finetune the model with n_epochs=1 and learning_rate_multiplier=0.03
# Set api key
#import os
#os.environ["OPENAI_API_KEY"] = api_key
!openai api fine_tunes.create -t "frichti_clean_prepared (2).jsonl" -m davinci --n_epochs 1 --learning_rate_multiplier 0.03

Upload progress: 100%|███████████████████████| 511k/511k [00:00<00:00, 230Mit/s]
Uploaded file from frichti_clean_prepared (2).jsonl: file-vOqOrqEBVXfAM2yLUIIo5KRA
Created fine-tune: ft-nOzzYuCuuZ1t4XPw7KcrzYgt
Streaming events until fine-tuning is complete...

(Ctrl-C will interrupt the stream, but not cancel the fine-tune)
[2023-02-17 13:55:54] Created fine-tune: ft-nOzzYuCuuZ1t4XPw7KcrzYgt
[2023-02-17 14:01:46] Fine-tune costs $5.35

Stream interrupted (client disconnected).
To resume the stream, run:

  openai api fine_tunes.follow -i ft-nOzzYuCuuZ1t4XPw7KcrzYgt



In [167]:
# cancel fine-tune ft-kK4Q9Mhp50YseXWvzldxFCCR
!openai api fine_tunes.cancel -i ft-kK4Q9Mhp50YseXWvzldxFCCR

{
  "created_at": 1676048480,
  "events": [
    {
      "created_at": 1676048480,
      "level": "info",
      "message": "Created fine-tune: ft-kK4Q9Mhp50YseXWvzldxFCCR",
      "object": "fine-tune-event"
    },
    {
      "created_at": 1676048574,
      "level": "info",
      "message": "Fine-tune cancelled",
      "object": "fine-tune-event"
    }
  ],
  "fine_tuned_model": null,
  "hyperparams": {
    "batch_size": null,
    "learning_rate_multiplier": 0.03,
    "n_epochs": 1,
    "prompt_loss_weight": 0.01
  },
  "id": "ft-kK4Q9Mhp50YseXWvzldxFCCR",
  "model": "davinci",
  "object": "fine-tune",
  "organization_id": "org-a1DhqGAsZUtMPiNrRMdwRfxf",
  "result_files": [],
  "status": "cancelled",
  "training_files": [
    {
      "bytes": 829852,
      "created_at": 1676048480,
      "filename": "frichti_clean_prepared (2).jsonl",
      "id": "file-otprMHq4RsdvdZmRDUrxtiOG",
      "object": "file",
      "purpose": "fine-tune",
      "status": "processed",
      "status_details": nu

In [6]:
import openai
api_key = ""
openai.api_key = api_key
import os
os.environ["OPENAI_API_KEY"] = api_key

In [8]:
!openai api fine_tunes.follow -i ft-nOzzYuCuuZ1t4XPw7KcrzYgt

[2023-02-17 13:55:54] Created fine-tune: ft-nOzzYuCuuZ1t4XPw7KcrzYgt
[2023-02-17 14:01:46] Fine-tune costs $5.35
[2023-02-17 14:01:47] Fine-tune enqueued
[2023-02-17 14:22:40] Fine-tune is in the queue. Queue number: 31
[2023-02-17 14:23:52] Fine-tune is in the queue. Queue number: 30
[2023-02-17 14:24:24] Fine-tune is in the queue. Queue number: 29
[2023-02-17 14:24:39] Fine-tune is in the queue. Queue number: 28
[2023-02-17 14:24:57] Fine-tune is in the queue. Queue number: 27
[2023-02-17 14:25:07] Fine-tune is in the queue. Queue number: 26
[2023-02-17 14:27:02] Fine-tune is in the queue. Queue number: 25
[2023-02-17 14:29:07] Fine-tune is in the queue. Queue number: 24
[2023-02-17 14:31:36] Fine-tune is in the queue. Queue number: 23
[2023-02-17 14:31:53] Fine-tune is in the queue. Queue number: 22
[2023-02-17 14:32:04] Fine-tune is in the queue. Queue number: 21
[2023-02-17 14:32:37] Fine-tune is in the queue. Queue number: 20
[2023-02-17 14:35:13] Fine-tune is in the queue. Queue

In [13]:
prompt = "Une newsletter de Frichti concernant une offre sur le vin rouge"

In [200]:
openai.api_key = api_key

In [14]:
#ft_model = 'davinci:ft-personal-2023-02-10-18-12-44'
ft_model = 'davinci:ft-personal-2023-02-17-14-12-03'

In [21]:
res = openai.Completion.create(model=ft_model, prompt=prompt + '\n\n###\n\n', max_tokens=2000, stop=['END'], temperature=0.7, top_p=1, best_of=1)


In [22]:
print(res["choices"][0]["text"])

 🔥🔥🔥🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷


In [ ]:
print(2)

: 